# SIT720 Assignment 3
Bryon Baker

Student number: 85031775

In [ ]:
# TOTO: Uncomment this before submitting the assignment

#!pip3 install numpy==1.22.3
#!pip3 install pandas== 1.4.2
!pip3 install scikit-learn==1.0.2
#!pip3 install statsmodels==0.13.2

In [ ]:
import warnings
import timeit
import random
import pandas as pd # dataframe manipulation
import numpy as np # linear algebra
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler 
# Configure ssl for unverified content so we can load a dataset from an unknown source (github).
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# Question 1

Load and explore the training dataset. Explain the steps that you have taken.

In [ ]:
import os

HOME = os.getenv('HOME')
print(HOME)

train_url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass3/ac_train_data.csv'
test_url = 'https://raw.githubusercontent.com/bryonbaker/datasets/main/SIT720/Ass3/ac_test_data.csv'

train_path = HOME+"/datasets/SIT720/Ass3/ac_train_data.csv"
test_path = HOME+"/datasets/SIT720/Ass3/ac_test_data.csv"

#
# Work out if the datasets are local. If not use a remote url. Preference is local.
#
print("Load datasets from local or remote resource:")
print("="*50)
if os.path.isfile(train_path):
    print(f"Training data is local: {train_path}")
    training_data = train_path
else:
    print("Training data is remote: {}".format(train_url))
    training_data = train_url

if os.path.isfile(test_path):
    print(f"Test data is local: {test_path}")
    test_data = test_path
else:
    print("Test data is remote: {}".format(test_url))
    test_data = test_url
print()

# Load the datasets from either local or remote.
eng_train_df = pd.read_csv(training_data)
eng_val_df = pd.read_csv(test_data)

In [ ]:
# Drop the columns not used in the assignment

# Drop the sequence number (column 0) from the test dataset. Column 0 is unlabeled so use the index number.
eng_train_df = eng_train_df.drop(eng_train_df.columns[[0]],axis=1)
eng_val_df = eng_val_df.drop(eng_val_df.columns[[0]],axis=1)

In [ ]:
# Ordinal encode the days of week.

oldCol = "dayofweek"
newCol = "dayofweeknum"
dayMap = {'Sun' : 0, 'Mon': 1, "Tue" : 2, "Wed" : 3, "Thu" : 4, "Fri" : 5, "Sat" : 6 }    # How to map the values

eng_train_df[newCol] = eng_train_df[oldCol].map(dayMap)    # Adds a new column with proper boolean values
eng_train_df = eng_train_df.drop([oldCol], axis=1)    # Drop the old column before renaming the new column to the name just dropped
eng_train_df = eng_train_df.rename(columns={newCol : oldCol})
      
eng_train_df.head()

In [ ]:
# Ordinal encode the dayof week in the test dataset.
eng_val_df[newCol] = eng_val_df[oldCol].map(dayMap)    # Adds a new column with proper boolean values
eng_val_df = eng_val_df.drop([oldCol], axis=1)    # Drop the old column before renaming the new column to the name just dropped
eng_val_df = eng_val_df.rename(columns={newCol : oldCol})
      
eng_val_df.head()

In [ ]:
# Split out the X and y from the datasets
eng_train_X = eng_train_df.drop(['ac'], axis=1)
eng_train_y = eng_train_df["ac"]
eng_val_X = eng_val_df.drop(['ac'], axis=1)
eng_val_y = eng_val_df['ac']

print("Training Dataset")
print(f"{eng_train_X.head()}\n")
print(f"{eng_train_y.head()}\n")

print("Testing Dataset")
print(f"{eng_val_X.head()}\n")
print(f"{eng_val_y.head()}\n")

In [ ]:
print("Summary Analysis of Dataset")
print(eng_train_X.describe())
print()

print("Summary Analysis of the Class")
print(eng_train_y.value_counts())

In [ ]:
print("Variable Correlation Matrix")
corr_X = eng_train_df.corr()
sn.set(rc = {'figure.figsize':(15,8)})
sn.heatmap(corr_X, annot=True)
plt.show()

In [ ]:
# Code snippet coutesy of: https://benjaminobi.medium.com/5-minutes-tutorial-on-how-to-compute-and-visualize-the-covariance-matrix-2597ab98d9ee#:~:text=The%20covariance%20matrix%20gives%20the,that%20displays%20the%20correlation%20coefficients.
#
stdsc = StandardScaler() 
X_std = stdsc.fit_transform(eng_train_df[eng_train_df.columns].iloc[:,range(0,len(eng_train_df.columns))].values)
cov_mat =np.cov(X_std.T)
plt.figure(figsize=(12,12))
sn.set(font_scale=1.5)
hm = sn.heatmap(cov_mat,
                 cbar=True,
                 annot=True,
                 square=True,
                 fmt='.2f',
                 annot_kws={'size': 12},
                 cmap='coolwarm',                 
                 yticklabels=eng_train_df.columns,
                 xticklabels=eng_train_df.columns)
plt.title('Covariance matrix with correlation coefficients', size = 18)
plt.tight_layout()
plt.show()

In [ ]:
# Plot an auto correlation to review the lag .
# A spike close to zero is evidence against autocorrelation
# A large spike is evidence of auto correlation
autocorrelation_plot(eng_train_df["load"])

There is a positive correlation within approximately the first 100,000 lags. 

The spikes at what appears to be < 1000 indicates that load is correlated for small windows of time. I.e. For a short period of time, when load is going up it continues to go up, and when load is going down it continues to go down.

Ref: https://www.dummies.com/article/technology/information-technology/data-science/big-data/autocorrelation-plots-graphical-technique-for-statistical-data-141241/


In [ ]:
# Generate a pairplot
# Commented out because this plot takes a VERY long time to run
# TODO: Uncomment before submitting assignment.
#sn.pairplot(eng_train_df[eng_train_X.columns], height=2.0)

### Visualise the relationship between load 

# Question 2
Analyse the importance of the feautues for predicting air conditioner status using two different approaches.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif

def plotFeaturePerformance( fs, ticks, title ):
    # what are scores for the features
    for i in range(len(fs.scores_)):
        print('Feature %d: %f' % (i, fs.scores_[i]))
        
    # plot the scores
    plt.bar([i for i in range(len(fs.scores_))], fs.scores_)
    plt.xlabel("Feature")
    plt.ylabel("K-Best Score")
    plt.title(label=title, loc="center")
    plt.xticks(np.arange(len(ticks)), labels=ticks, rotation=70)
    
    plt.show()

def selectFeatures( X_train, y_pred, criteria, num_features ):
    fs = SelectKBest(criteria, k=num_features)
    fs.fit(X_train,y_pred)
    new_x_train = fs.transform(X_train)
    
    return new_x_train, fs

In [ ]:
# Define the number of features to find.
num_features = 5
print("Column names:\n{}".format(eng_train_X.columns))
new_x_train1, fs1 = selectFeatures( eng_train_X, eng_train_y, f_classif, num_features )
new_x_train2, fs2 = selectFeatures( eng_train_X, eng_train_y, f_regression, num_features )

In [ ]:
# Plot the feature selection charts.
plotFeaturePerformance( fs1, eng_train_X.columns, "ANOVA Feature Selection" )

In [ ]:
plotFeaturePerformance( fs2, eng_train_X.columns, "Regression Feature Selection" )

In [ ]:
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
Xnorm = scale(eng_train_X)

# Perform the Principal Component Analysis. The number of components is the number of coumns in the dataset.
# The number of components is the minimum of the number of data elements and festures 
n_components=min(eng_train_X.shape[0], eng_train_X.shape[1])
print("Number of comoponets: {}".format(n_components))
n_components=7
pca = PCA(n_components)
Xnew = pca.fit_transform(Xnorm)
print("PCA details => {}".format(pca))

print(pca.get_params())

print('='*50)
print("Explained Variance:")
# Calculate the amount of variance explained by each PC
var= pca.explained_variance_ratio_
print(var)

print('='*50)
print("Cumulative Explained Variance:")

#cum_var=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
cum_var=np.cumsum(pca.explained_variance_ratio_*100)
print(cum_var)

dimensions = {}

# Find the minimum dimensions that capture 85% and 95% respectively
num_dimensions = 0
for value in cum_var:
    if value >= 85:
        dimensions[value]=num_dimensions
        break
    num_dimensions += 1

num_dimensions = 0
for value in cum_var:
    if value >= 95:
        dimensions[value]=num_dimensions
        break
    num_dimensions += 1

print('='*50)

In [ ]:
pca_num = a_list = list(range(1, n_components+1))
plt.plot(pca_num, cum_var, marker='.')
plt.xlabel("Principal components")
plt.ylabel("Variance captured")
plt.title(label="Captured Variance vs. Latent Dimensionality", loc="center")
plt.show()

for key,value in dimensions.items():
    print("The minimum dimension that captures at least {:.0f}% variance is: {}".format(key,value))

In [ ]:
print(pca.components_)
print(type(pca.components_))
pca.components_.shape

In [ ]:
n_groups = pca.components_.shape[1]

fig, ax = plt.subplots()
index = np.arange(n_groups)
bar_width = 0.35

plt.bar(index, np.absolute(pca.components_[0,:]), 
        bar_width,
        color='r',
        label='Component 1')

plt.xlabel('Magnitude')
plt.ylabel('Feature number')
plt.title('Principal Component Scores')
#plt.xticks(index + bar_width / 2, ('A', 'B', 'C', 'D', 'E'))
plt.xticks(index+bar_width/2, eng_train_X.columns)
plt.legend()

plt.tight_layout()
plt.show()

## Feature Engineering
Now that we have analysed the features and their importance we will create a new training dataset that has the features we want in the final train/test dataset and a validation dataset.

In [ ]:
train_X = eng_train_X
train_y = eng_train_y
val_X = eng_val_X
val_y = eng_val_y

In [ ]:
train_X.shape[0]

# Question 3
Based on the training data, create three different supervised machine learning models excet any ensemble approach for predicting air conditioner status
   1.	Report performance score using a suitable metric on the test data. Is it possible that the presented result is an underfitted or overfitted one? Justify.  
   2.	Justify different design decisions for each ML model used to answer this question.
   3.	Have you optimised any hyper-parameters for each ML model? What are they? Why have you done that? Explain. 
   4.	Finally, make a recommendation based on the reported results and justify it.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [ ]:
import timeit

def current_milli_time():
    return round(time.time() * 1000)

In [ ]:
from sklearn import metrics

def display_stats(y_test, y_pred):
    # Model Accuracy: how often is the classifier correct?
    print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

    # Model Precision: what percentage of positive tuples are labeled as such?
    print("Precision:",metrics.precision_score(y_test, y_pred))

    # Model Recall: what percentage of positive tuples are labelled as such?
    print("Recall:",metrics.recall_score(y_test, y_pred))

    #Calculate F1 Score
    print("F1 Score:",metrics.f1_score(y_test, y_pred))

    #Calculate Mean Absolute Error
    print("Mean Absolute Error:",metrics.mean_absolute_error(y_test, y_pred))

    # kappa
    print("Cohens kappa:", metrics.cohen_kappa_score(y_test, y_pred))

    # ROC AUC
    print("ROC AUC:", metrics.roc_auc_score(y_test, y_pred))

In [ ]:
def validate_model( model, validation_dataset_X, validation_dataset_y ):
    print(f"Validating model: {model}")
    y_pred = model.predict( validation_dataset_X )
    # Find the performance info
    accuracy = metrics.accuracy_score(validation_dataset_y, y_pred)
    precision = metrics.precision_score(validation_dataset_y, y_pred)
    recall = metrics.recall_score(validation_dataset_y, y_pred)
    f1 = metrics.f1_score(validation_dataset_y, y_pred)

    print(f"Accuracy {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}")

In [ ]:
import warnings
# Some of the hyper parameter combinations are invalid. Supress the warnings.
warnings.filterwarnings('ignore')

### Test using the Decision Tree Model
The decision tree is a fast and explainable model. The most common hyperparameters to tune are the max leaf nodes, criterion and the minimum samples before you can split.

In [ ]:
def perform_decision_tree( df_X, df_y, c, m, s ):
    skf = StratifiedKFold(n_splits=10)
    score = []
    for train_index, test_index in skf.split(df_X, df_y):
        X_tr = df_X.iloc[train_index]
        X_val = df_X.iloc[test_index]
        y_tr = df_y.iloc[train_index]
        y_val = df_y.iloc[test_index]

        # Train the model with the training split
        model = DecisionTreeClassifier(criterion = c, max_leaf_nodes=m, min_samples_split=s)
        start = timeit.default_timer()
        model.fit(X_tr, y_tr)
        stop = timeit.default_timer()
        train_time = stop - start

        # Now work with the testing split (vaidation set) and measure the performance
        y_pred = model.predict(X_val)

        # Find the performance info
        accuracy = metrics.accuracy_score(y_val, y_pred)
        precision = metrics.precision_score(y_val, y_pred)
        recall = metrics.recall_score(y_val, y_pred)
        f1 = metrics.f1_score(y_val, y_pred)

        # Store the results so we can average them after all the splits have been run.
        score.append([accuracy, precision, recall, f1, train_time])
        # print(f"{[accuracy, precision, recall, f1, train_time]}")
    return model, score

In [ ]:
max_leaf_nodes = [10, 50, 100, 500]
criterion = ["gini", "entropy"]
min_samples_split = [2,3,4,8]

score = []
avg_score= []
for c in criterion:
    for m in max_leaf_nodes:     
        for s in min_samples_split:   
            _, score = perform_decision_tree( train_X, train_y, c, m, s )

            # print(f"score: {score}")
            averages = np.array(score).mean(axis=0)
            
            print(f"{c} {m} {s}: Averages: {averages}")

In [ ]:
model, score = perform_decision_tree( train_X, train_y, 'gini', 500, 2 )
print(f"Trained results: {np.array(score).mean(axis=0)}")
print("Validating model with validation dataset:")
validate_model( model, val_X, val_y )

### Test Logistic Regression Model

In [ ]:
def perform_logistic_regression(df_X, df_y, p, s, c):
    skf = StratifiedKFold(n_splits=10)
    score = []
    for train_index, test_index in skf.split(df_X, df_y):
        X_tr = df_X.iloc[train_index]
        X_val = df_X.iloc[test_index]
        y_tr = df_y.iloc[train_index]
        y_val = df_y.iloc[test_index]

        # Train the model with the training split
        model = LogisticRegression(max_iter=10000, penalty=p, solver=s, C=int(c))
        start = timeit.default_timer()
        model.fit(X_tr, y_tr)
        stop = timeit.default_timer()
        train_time = stop - start

        # Now work with the testing split (vaidation set) and measure the performance
        y_pred = model.predict(X_val)

        # Find the performance info
        accuracy = metrics.accuracy_score(y_val, y_pred)
        precision = metrics.precision_score(y_val, y_pred)
        recall = metrics.recall_score(y_val, y_pred)
        f1 = metrics.f1_score(y_val, y_pred)

        # Store the results so we can average them after all the splits have been run.
        score.append([accuracy, precision, recall, f1, train_time])
        #print(f"{[accuracy, precision, recall, f1]}")

    return model, score

In [ ]:
penalty = ['none','l2']     # have left out none because of feedback form previous runs and the fact it generates a ton of incompatable param warnings
solver = ['newton-cg', 'lbfgs', 'liblinear']
C = [100, 10, 1.0, 0.1, 0.01]

i = 1
avg_score= []
for p in penalty:
    for s in solver:
        for c in C:
            if p == 'none' and s == 'liblinear':
                continue        # These are incompatible hyperparameters so skip them
            if p == 'l2' and c < 1.0:
                continue        # These are incompatible hyperparameters so skip them
             
            _, score = perform_logistic_regression( train_X, train_y, p, s, c )

            averages = np.array(score).mean(axis=0)
            
            print(f"Averages: penalty: {p} solver: {s} C: {c} {averages}")
            
            #avg_score.append([p, s, c, averages])
            #print(f"\niteration: {i}: {avg_score}")
            i += 1
print("complete:")

In [ ]:
model, score = perform_logistic_regression( train_X, train_y, 'l2', 'lbfgs', 1.0 )
print(f"Trained results: {np.array(score).mean(axis=0)}")
print("Validating model with validation dataset:")
validate_model( model, val_X, val_y )

### Test the SVM Model

In [ ]:
def perform_svm(df_X, df_y, c):
    skf = StratifiedKFold(n_splits=10)
    score = []
    for train_index, test_index in skf.split(df_X, df_y):
        X_tr = df_X.iloc[train_index]
        X_val = df_X.iloc[test_index]
        y_tr = df_y.iloc[train_index]
        y_val = df_y.iloc[test_index]

        # Train the model with the training split
        model = SVC(C=c)
        start = timeit.default_timer()
        model.fit(X_tr, y_tr)
        stop = timeit.default_timer()
        train_time = stop - start

        # Now work with the testing split (vaidation set) and measure the performance
        y_pred = model.predict(X_val)

        # Find the performance info
        accuracy = metrics.accuracy_score(y_val, y_pred)
        precision = metrics.precision_score(y_val, y_pred)
        recall = metrics.recall_score(y_val, y_pred)
        f1 = metrics.f1_score(y_val, y_pred)

        # Store the results so we can average them after all the splits have been run.
        score.append([accuracy, precision, recall, f1, train_time])
        print(f"{[accuracy, precision, recall, f1, train_time]}")
        
    return model, score

## Tune the Selected Model
Now we have an idea of the best model algorithm, a benchmark of performance. Drop some of the less-important features, retrain the model and test the performance.

In [ ]:
# Now we have an idea of the best model algorithm, a benchmark of performance. Drop some of the less-important features, retrain the model and test the performance.
train_X = eng_train_X
train_X = train_X.drop(['dif','nonlinear','hurst','dayofweek'], axis=1)
print(train_X.head)

#### Train the Model

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier

skf = StratifiedKFold(n_splits=10)

max_leaf_nodes = [500]
criterion = ["gini"]
min_samples_split = [2]

score = []
avg_score= []
for c in criterion:
    for m in max_leaf_nodes:     
        for s in min_samples_split:  
            for train_index, test_index in skf.split(train_X, train_y):
                X_tr = train_X.iloc[train_index]
                X_val = train_X.iloc[test_index]
                y_tr = train_y.iloc[train_index]
                y_val = train_y.iloc[test_index]

                # Train the model with the training split
                model = DecisionTreeClassifier(criterion = c, max_leaf_nodes=m, min_samples_split=s)
                start = timeit.default_timer()
                model.fit(X_tr, y_tr)
                stop = timeit.default_timer()
                train_time = stop - start

                # Now work with the testing split (vaidation set) and measure the performance
                y_pred = model.predict(X_val)

                # Find the performance info
                accuracy = metrics.accuracy_score(y_val, y_pred)
                precision = metrics.precision_score(y_val, y_pred)
                recall = metrics.recall_score(y_val, y_pred)
                f1 = metrics.f1_score(y_val, y_pred)

                # Store the results so we can average them after all the splits have been run.
                score.append([accuracy, precision, recall, f1, train_time])

            averages = np.array(score).mean(axis=0)
            
            print(f"{c} {m} {s}: Averages: {averages}")
            
av_df = pd.DataFrame(data=averages, index = ["Accuracy", "Precision", "Recall", "F1", "Train time"])

#### Test the New Model With the Validation Set
Using the model trained in the previous cell, perform a predict using the assignment validation dataset

In [ ]:
validate_model( model, val_X, val_y )
